In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2"

import evaluate
import transformers
import argparse
import torch
import json
import datasets

from numpy import mean
from tqdm import tqdm

TOKEN_QUESTION = "### Question:"
TOKEN_END_QUESTION = ""
TOKEN_CONTEXT = "### Context:"
TOKEN_END_CONTEXT = ""
TOKEN_ANSWER = "### Answer:"
TOKEN_END_ANSWER = ""
HIGHLIGHT_ANSWER = ""
SPLIT_SEED = 42
NPROC = 32
HIGHLIGHT = True
LLAMA_PATH = "./results/llama-7B/checkpoint-1000"


/storagenfs/l.miglior/answer-aware-question-generation/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
bertscore = evaluate.load("bertscore")
rouge = evaluate.load("rouge")

In [4]:
tokenizer = transformers.AutoTokenizer.from_pretrained(LLAMA_PATH)
model = transformers.AutoModelForCausalLM.from_pretrained(LLAMA_PATH, device_map="auto")

Loading checkpoint shards: 100%|██████████| 2/2 [00:19<00:00,  9.75s/it]


In [5]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

In [3]:

def get_inputs_target(e):
    answer_start = e["answers"]["answer_start"][0]
    # add highlight token to context
    ans_len = len(e["answers"]["text"][0])

    e["context"] = (
        e["context"][:answer_start]
        + " "
        + HIGHLIGHT_ANSWER
        + " "
        + e["context"][answer_start : answer_start + ans_len]
        + " "
        + HIGHLIGHT_ANSWER
        + " "
        + e["context"][answer_start + ans_len :]
        + " "
        + TOKEN_ANSWER
        + " "
        + e["answers"]["text"][0]
        + " "
        + TOKEN_END_ANSWER
    )

    return {
        # answer + context + question for causal language modeling
        "text": f'<s> {e["context"]} {TOKEN_QUESTION}',
    }



def preprocess_squad_dataset(dataset_name="squad", split="train"):
    dataset = datasets.load_dataset(dataset_name, split=split).select(range(1000,2000))
    dataset = dataset.map(get_inputs_target, num_proc=NPROC)
    return dataset


# load dataset
valid_dataset = preprocess_squad_dataset(dataset_name="squad", split="validation")

Map (num_proc=32): 100%|██████████| 1000/1000 [00:00<00:00, 1787.43 examples/s]


In [4]:
shuffled = valid_dataset.shuffle(SPLIT_SEED).select(range(100))


In [12]:
# put all contexts, questions, and references answers in a json file
with open("llama.json", "w") as f:
    for e in shuffled:
        f.write(
            json.dumps(
                {
                    "context": e["context"],
                    "question": e["question"],
                    "answers": e["answers"]["text"][0],
                }
            )
            + ",\n"
        )

In [21]:
example = valid_dataset[2]
sequences = pipeline(
            example['text'],
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id,
            max_length=512
        )
prediction = str([s["generated_text"] for s in sequences])
print("> Prediction:", prediction.split(TOKEN_QUESTION)[1].split('?')[0], "\n", "> Original:", example["question"], "\n") 

> Prediction:  What is the name of the big park in the northern Mokotów 
 Original: Where was the first horse racetrack located? 



In [30]:
predictions = []
references = []
contexes = []

for examples in tqdm(valid_dataset.shuffle(SPLIT_SEED).select(range(100))):
    sequences = pipeline(
            examples['text'],
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id,
            max_length=512
        )
    prediction = str([s["generated_text"] for s in sequences])
    predictions.append(prediction.split(TOKEN_QUESTION)[1].split('?')[0] + "?")
    references.append(examples["question"])
    contexes.append(examples["context"])
    print("> Prediction:", prediction.split(TOKEN_QUESTION)[1].split('?')[0], "\n" + "> Original:", examples["question"], "\n") 

  0%|          | 0/100 [00:00<?, ?it/s]

/storagenfs/l.miglior/answer-aware-question-generation/.venv/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
  1%|          | 1/100 [00:27<45:57, 27.85s/it]

> Prediction:  What is the main problem with modern schooling 
> Original: What do critics think the cause of problems with modern schooling is? 



  2%|▏         | 2/100 [00:55<45:12, 27.68s/it]

> Prediction:  What is the most significant difference between primary school and secondary school teaching 
> Original: What's the biggest difference in the teaching relationship for primary and secondary school? 



...

> Prediction:  What did Tesla believe X-rays to be 
> Original: What did tesla incorrectly believe about x-rays? 



 99%|█████████▉| 99/100 [40:32<00:21, 21.19s/it]

> Prediction:  Who was awarded the contract to build the AC distribution system at the Niagara Falls 
> Original: What company was chosen to build an AC distribution system at Niagara Falls? 



100%|██████████| 100/100 [40:54<00:00, 24.54s/it]

> Prediction:  What is the main focus of complexity theory 
> Original: Complexity theory classifies problems based on what primary attribute? 



In [39]:
import numpy as np
np.mean(bertscore.compute(predictions=predictions, references=references, lang="en")['f1'])

0.9142226606607438

In [33]:
# save scores
metrics = {
    "bertscore": bertscore.compute(predictions=predictions, references=references, lang="en"),
    "rouge": rouge.compute(predictions=predictions, references=references),
    "predictions": predictions,
    "references": references,
    "contexes": contexes
}

with open("results/llama-7B/eval.json", "w") as f:
    json.dump(metrics, f)